In [213]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import bellman_ford
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import re
import networkx as nx
from openpyxl import load_workbook

In [817]:
data = pd.read_excel("data/1.2.24.xlsx")
date = str(data["Date"][0].month) + "/" + str(data["Date"][0].day) + "/" + str(data["Date"][0].year)

tokens = []
for elt in data['Pool Tokens']:
    res = re.split(' / ', elt)
    if res[0] not in tokens:
        tokens.append(res[0])
    if res[1] not in tokens:
        tokens.append(res[1])
        
l = len(tokens)

In [818]:
# graph=pd.DataFrame(np.zeros((l,l)),columns=tokens,index=tokens)
# for index, row in data[['Pool Tokens', 'Volume Token 0 (Excel)', 'Volume Token 1 (Excel)', 'Volume USD (Excel)', 'Fees USD (Excel)']].iterrows():
#     token1, token2 = re.split(' / ', row[0])
#     volume1 = row[1]
#     volume2 = row[2]
#     if volume1 == volume2 and volume1 !=0 and volume2 !=0:
#         raise Exception("EQUALITY")
#     if volume1 != 0 and volume2 !=0:
#         if row[3] !=0:
#             feetier = row[4]/row[3]
#         else:
#             feetier = 0.003
#         #print(feetier)
#         graph.loc[token1, token2] = -np.log(volume2/volume1 * (1-feetier))
#         graph.loc[token2, token1] = -np.log(volume1/volume2 * (1-feetier))

In [819]:
# graph = [
# [0, np.log(9/16) ,np.log(18/25)],
# [np.log(16/9), 0, np.log(32/25)],
# [np.log(25/18), np.log(25/32), 0]
# ] -3.489095140951183
# graph = csr_matrix(graph)
# print(graph)

In [820]:
#dist_matrix, predecessors = bellman_ford(csgraph=graph, directed=True, return_predecessors=True)

In [821]:
#dist_matrix

In [822]:
G = nx.DiGraph()
for elt in tokens:
    G.add_node(elt)
for index, row in data[['Pool Tokens', 'Volume Token 0 (Excel)', 'Volume Token 1 (Excel)', 'Volume USD (Excel)', 'Fees USD (Excel)']].iterrows():
    token1, token2 = re.split(' / ', row[0])
    volume1 = row[1]
    volume2 = row[2]
    if volume1 == volume2 and volume1 !=0 and volume2 !=0:
        raise Exception("EQUALITY")
    if volume1 != 0 and volume2 !=0:
        if row[3] !=0:
            feetier = row[4]/row[3]
        else:
            feetier = 0.003
        #print(feetier)
        G.add_weighted_edges_from( [(token1, token2, -np.log(volume2/volume1 * (1-feetier)))] )
        G.add_weighted_edges_from( [(token2, token1, -np.log(volume1/volume2 * (1-feetier)))] )

In [823]:
def find_cycle(G, tokens):
    for t in tokens:
        try:
            nx.find_negative_cycle(G, t)
        except Exception:
            pass
        else:
            return nx.find_negative_cycle(G, t)
find_cycle(G, tokens)

['WETH', 'WETH']

In [805]:
# WRITE PROGRAM THAT NUMERICALLY CALCULATES HOW MUCH YOU GAINED IN ARBITRAGE

In [806]:
def uniswap_rate(A,B,alpha):
    return B/(A*(1+alpha/A)**2)

def revenue(q, cycle, dataset):
    pools = []
    for i in range(len(cycle)-1):
        token1 = cycle[i]
        token2 = cycle[i+1]
        if dataset.isin([token1 + " / " + token2]).any().any():
            pool =  dataset.loc[(dataset["Pool Tokens"] == token1 + " / " + token2)].iloc[-1]
            q1 = pool["Volume Token 0 (Excel)"]
            q2 = pool["Volume Token 1 (Excel)"]
        else:
            pool =  dataset.loc[(dataset["Pool Tokens"] == token2 + " / " + token1)].iloc[-1]
            q2 = pool["Volume Token 0 (Excel)"]
            q1 = pool["Volume Token 1 (Excel)"]
        
        volume = pool['Volume USD (Excel)']
        fees = pool['Fees USD (Excel)']
        
        if volume != 0:
            feetier = fees/volume
        else:
            feetier = 0.003
        
        
        pools.append([q1,q2,feetier])
    alpha = q
    for [q1,q2,feetier] in pools:
        if alpha > q1:
            return np.inf
        beta = (1-feetier) * alpha * uniswap_rate(q1,q2,alpha)
        alpha = beta
    return -beta

In [807]:
cycle = find_cycle(G, tokens)

def prof(q):
    return revenue(q, cycle, data)+q

m = -minimize(prof, [0.0000001]).fun
print(minimize(prof, [0.0000001]))

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -0.00022597732552447937
        x: [ 1.176e-02]
      nit: 2
      jac: [ 4.054e-07]
 hess_inv: [[ 3.040e-01]]
     nfev: 8
     njev: 4


In [808]:
filename = "Profits.xlsx"
workbook = load_workbook(filename)
worksheet = workbook.active
data = {'Quantity': [m], 'Security': [cycle[0]], 'Date': [date]}
for record in list(zip(*data.values())):
    worksheet.append(record)
    
workbook.save(filename)

In [809]:
m

0.00022597732552447937